In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/data.csv')
data.head()

drop_columns = ['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1']
data = data.drop(columns=drop_columns)

In [3]:
data.describe()

,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
count,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,15821472.0,1.582147e+07
mean,8.152679e+01,3.816483e+02,6.238540e-01,1.840605e+00,4.346986e+00,1.0,2.668835e-02
std,1.632448e+02,3.959386e+02,9.249152e+00,1.530005e+00,4.317701e+00,0.0,1.611710e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00
25%,1.400000e+01,5.200000e+01,0.000000e+00,1.000000e+00,0.000000e+00,1.0,0.000000e+00
50%,1.900000e+01,2.170000e+02,0.000000e+00,2.000000e+00,4.000000e+00,1.0,0.000000e+00
75%,6.000000e+01,6.110000e+02,0.000000e+00,3.000000e+00,7.000000e+00,1.0,0.000000e+00
max,3.443000e+03,1.632000e+03,8.290000e+02,1.000000e+01,1.600000e+01,1.0,1.000000e+00


In [4]:
def analysis(data: pd.DataFrame):
    # По data.describe заметим, что все значения impressions = 1, значит колонка бесполезна
    # Посчитаем в каких временных интервалах лежат наши данные
    print("Dates:", sorted(set(data['date_time'].map(lambda s: s[:10]))))
    # Заметим лишний день в нашей неделе - 1 сентября
    
    # Посмотрим на пропущенные значения
    print("Count of null data: ", data.isnull().sum().sum())
    # Пропущенных данных нет
    
    # Проверим, что все категориальные фичи представленны в достаточном количестве
    print(data['os_id'].value_counts())
    # Заметим, что некоторые os-и не популярны: 8, 7, 9, 10. Объединим их в одну группу
    print(data['country_id'].value_counts())
    # Страны представлены в достаточном количестве
    print(data['zone_id'].value_counts())
    # Среди мест баннера есть непопулярные, закодируем их в одну группу
    print(data['banner_id'].value_counts())
    # Среди банеров есть как очень популярные, по которым много информации, также много тех, что встречаются 1 раз во всем датасете, то есть не несут в себе полезной информации


analysis(data)

Dates: ['2021-09-01', '2021-09-26', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-02']
Count of null data:  0
2     4589979
0     3856798
1     3178693
4     3012541
3      759767
6      310346
5      111946
8        1273
7          99
9          25
10          5
Name: os_id, dtype: int64
0     4956393
5     1910054
6     1582705
7     1525569
1     1346321
3     1337392
15     603301
12     478038
9      439004
4      388801
10     356490
11     212245
8      210211
13     190855
14     138385
16      79166
2       66542
Name: country_id, dtype: int64
17      2280422
14      1638642
12       736352
0        708379
19       693292
         ...   
2186          1
2188          1
2518          1
2962          1
3443          1
Name: zone_id, Length: 3444, dtype: int64
22      613367
361     387563
3       286999
18      262946
21      246378
         ...  
1501         1
1503         1
1504         1
1507         1
1632         1
Name: banner_id, Length: 

In [5]:
from sklearn.preprocessing import OneHotEncoder

def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    # Удаляем 1е сентября
    data.drop(data[data['date_time'].map(lambda s: s[:10]) == '2021-09-01'].index, inplace=True, axis=0)
    # Преобразуем дату в datetime для дальнейших манипуляций
    data['datetime'] = pd.to_datetime(data['date_time'])
    # Добавим фичи, говорящие о дне недели (поведение пользователей в рабочие дни могут отличаться от выходных)
    data["day_of_week"] = data["datetime"].dt.dayofweek
    # Добавим фичи, времени суток
    data['hour'] = data["datetime"].dt.hour
    # Удаляем непопулярные значения
    zone_mask = data['zone_id'].isin(data.groupby('zone_id')['date_time'].count().sort_values(key=lambda x: -x).index[250:])
    data.loc[zone_mask, 'zone_id'] = 0
    os_mask = data['os_id'].isin(data.groupby('os_id')['date_time'].count().sort_values(key=lambda x: -x).index[7:])
    data.loc[os_mask, 'os_id'] = 0
    banner_mask = data['banner_id'].isin(data.groupby('banner_id')['date_time'].count().sort_values(key=lambda x: -x).index[600:])
    data.loc[banner_mask, 'banner_id'] = 0
    categorical_cols = ["zone_id", "os_id", "day_of_week", "hour", "country_id", "banner_id"]
    # Понадобится в будущем для кросс-валидации
    data = data.sort_values("datetime")
    return categorical_cols, data

categorical_cols, df = feature_engineering(data)

In [6]:
def train_test_split(data: pd.DataFrame, categorical_cols):
    train_condition = data['date_time'].apply(lambda s: s.split()[0]) < '2021-10-02'
    test_condition = data['date_time'].apply(lambda s: s.split()[0]) == '2021-10-02'
    train_data = data[train_condition]
    test_data = data[test_condition]

    train_val = train_data['clicks']
    test_val = test_data['clicks']
    
    train_data = train_data.drop(['clicks', 'date_time', 'datetime'], axis=1)
    test_data = test_data.drop(['clicks', 'date_time', 'datetime'], axis=1)
    
    # Сразу создаем sparse-матрицы, так как в категориальных фичах будет много 0
    ohe = OneHotEncoder(handle_unknown='ignore', sparse = True)
    ohe.fit(train_data[categorical_cols])
    # Encoding задаем только на train, но значения кодируем везде
    train_data = ohe.transform(train_data[categorical_cols])
    test_data = ohe.transform(test_data[categorical_cols])

    return train_data, train_val, test_data, test_val

train_data, train_val, test_data, test_val = train_test_split(df, categorical_cols)

In [7]:
train_data, train_val, train_val.sum(), test_data, test_val, test_val.sum()

(<13692493x900 sparse matrix of type '<class 'numpy.float64'>'
 	with 82154958 stored elements in Compressed Sparse Row format>,
 8423230     0
 7683145     0
 4647065     0
 11536774    0
 1442602     0
            ..
 2876844     0
 10358819    0
 15652742    0
 14917859    0
 13364121    0
 Name: clicks, Length: 13692493, dtype: int64,
 346887,
 <2128978x900 sparse matrix of type '<class 'numpy.float64'>'
 	with 10514650 stored elements in Compressed Sparse Row format>,
 9767447     0
 13846765    0
 3091651     0
 10045990    0
 9054327     0
            ..
 1745969     0
 13959634    0
 13319080    0
 3336944     0
 3226555     0
 Name: clicks, Length: 2128978, dtype: int64,
 75362)

In [8]:
def create_model():
    return LogisticRegression(solver='liblinear', random_state=42, penalty="l2")

In [12]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression

def cv(train_data, train_val):
    model = create_model()
    param_search = {'C' : [0.001, 0.01, 0.1]}
    
    # TimeSeriesSplit чтобы тестовые данные были позже обучающих(иначе получится, что учимся на данных, которые учитывают вердикт)
    splits = TimeSeriesSplit(n_splits=3)
    gsearch = GridSearchCV(estimator=model, cv=splits,
                            param_grid=param_search, verbose=2, scoring=["neg_log_loss", "roc_auc"], refit="neg_log_loss")
    gsearch.fit(train_data, train_val)

    print(gsearch.cv_results_)
    # выбираем лучшую по logloss model
    best_model = gsearch.best_estimator_
    print("Best score: {} \nBest params: {}".format(gsearch.best_score_, gsearch.best_params_))
    
    return best_model.fit(train_data, train_val)

model = cv(train_data, train_val)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ............................................C=0.001; total time=   7.7s
[CV] END ............................................C=0.001; total time=  15.1s
[CV] END ............................................C=0.001; total time=  26.6s
[CV] END .............................................C=0.01; total time=  11.7s
[CV] END .............................................C=0.01; total time=  26.5s
[CV] END .............................................C=0.01; total time=  38.7s
[CV] END ..............................................C=0.1; total time=  16.6s
[CV] END ..............................................C=0.1; total time=  48.0s
[CV] END ..............................................C=0.1; total time= 1.2min
{'mean_fit_time': array([14.55146027, 23.78637997, 44.61973286]), 'std_fit_time': array([ 7.6981539 , 11.01271569, 23.72757774]), 'mean_score_time': array([1.93299039, 1.83300289, 1.87705056]), 'std_score_time': 

In [13]:
from sklearn.metrics import log_loss, roc_auc_score
import numpy as np

predictions = model.predict_proba(test_data)
baseline = np.full(test_val.shape, np.mean(test_val))

print('Model: ', roc_auc_score(test_val, model.predict_proba(test_data)[:, 1]), log_loss(test_val, model.predict_proba(test_data)))
print('Random: ', roc_auc_score(test_val, baseline), log_loss(test_val, baseline))

Model:  0.7754759364636382 0.13552848196046108
Random:  0.5 0.15303289904918688
